In [1]:
# https://www.kaggle.com/kernels/scriptcontent/11511967/notebook

import string
import numpy as np
import pandas as pd
# from IPython.display import display
from tqdm import tqdm
from collections import Counter
import ast
import re
from pprint import pprint

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
# import matplotlib.mlab as mlab
import seaborn as sb

from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer
# from textblob import TextBlob
import scipy.stats as stats

from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.manifold import TSNE

import nltk
from nltk.tag import tnt
from nltk.corpus import stopwords
from nltk.corpus import indian
from nltk.tokenize import word_tokenize,sent_tokenize

import gensim
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import gensim.corpora as corpora
from gensim.summarization import summarize
from gensim.test.utils import datapath

import stanfordnlp
import networkx as nx
import math

import warnings
warnings.filterwarnings("ignore")

# from bokeh.plotting import figure, output_file, show
# from bokeh.models import Label
# from bokeh.io import output_notebook
# output_notebook()

%matplotlib inline

/Users/rohanraj/opt/anaconda3/lib/python3.7/site-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Iterable
/Users/rohanraj/opt/anaconda3/lib/python3.7/site-packages/past/builtins/misc.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Mapping
/Users/rohanraj/opt/anaconda3/lib/python3.7/site-packages/nltk/decorators.py:68: DeprecationWarning: `formatargspec` is deprecated since Python 3.5. Use `signature` and the `Signature` object directly
  regargs, varargs, varkwargs, defaults, formatvalue=lambda value: ""
/Users/rohanraj/opt/anaconda3/lib/python3.7/site-packages/nltk/lm/counter.py:15: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, an

In [2]:
nlp = stanfordnlp.Pipeline ( lang = 'hi' ) 

Use device: cpu
---
Loading: tokenize
With settings: 
{'model_path': '/Users/rohanraj/stanfordnlp_resources/hi_hdtb_models/hi_hdtb_tokenizer.pt', 'lang': 'hi', 'shorthand': 'hi_hdtb', 'mode': 'predict'}
---
Loading: pos
With settings: 
{'model_path': '/Users/rohanraj/stanfordnlp_resources/hi_hdtb_models/hi_hdtb_tagger.pt', 'pretrain_path': '/Users/rohanraj/stanfordnlp_resources/hi_hdtb_models/hi_hdtb.pretrain.pt', 'lang': 'hi', 'shorthand': 'hi_hdtb', 'mode': 'predict'}
---
Loading: lemma
With settings: 
{'model_path': '/Users/rohanraj/stanfordnlp_resources/hi_hdtb_models/hi_hdtb_lemmatizer.pt', 'lang': 'hi', 'shorthand': 'hi_hdtb', 'mode': 'predict'}
Building an attentional Seq2Seq model...
Using a Bi-LSTM encoder
Using soft attention for LSTM.
Finetune all embeddings.
[Running seq2seq lemmatizer with edit classifier]
---
Loading: depparse
With settings: 
{'model_path': '/Users/rohanraj/stanfordnlp_resources/hi_hdtb_models/hi_hdtb_parser.pt', 'pretrain_path': '/Users/rohanraj/stanford

In [3]:
datafile = '../data/hindi.txt'
with open(datafile,'r',encoding='utf-8') as f:
    text = f.read()
    text = text.split("॥")

In [4]:
text_sentence = []
i = 0
for word in tqdm(text):
#     print(i,word)
    if re.findall("^\s?\d+\s?",word):
        continue
    else:   
        text_sentence.append(word)

100%|██████████| 1290/1290 [00:00<00:00, 407981.61it/s]


In [5]:
text_sentence[0:2]

[' वानर हनुमान सीमा के देखने की उत्कण्ठा येसूयास्त की प्रतीक्षा करता भया ',
 ' घ४-९) सीता को रावण के भन्तापुर में ढूंढना : वह महावीर्य महान हृदय पाछा वानर श्रेष्ठ रात के समय अद्वार से कोट को फांद कर का में प्रविष्ट हृया ']

In [6]:
len(text_sentence) #total sentences

667

In [7]:
# hindi stopwords
stop_words_df = pd.read_csv("../data/stopwords.txt", header = None)
stop_words = list(set(stop_words_df.values.reshape(1,-1).tolist()[0]))
stop_words.extend(["।", "।।", ")", "(", ",",'"',"हे", "हो", 'में','से','COMMA'])

In [8]:
len(stop_words)

723

In [9]:
stop_words[:2]

['दीजिए', 'तक']

In [10]:
def create_str_from_list(original_read_text):
    prepared_text = ""
    for line in original_read_text:
        line = line.split()
#       print(line)
        tmp_line = " ".join(line)
        prepared_text += " \n"+tmp_line
    return prepared_text

In [11]:
str_text = create_str_from_list(text_sentence)

In [12]:
str_text[0:2500]

' \nवानर हनुमान सीमा के देखने की उत्कण्ठा येसूयास्त की प्रतीक्षा करता भया \nघ४-९) सीता को रावण के भन्तापुर में ढूंढना : वह महावीर्य महान हृदय पाछा वानर श्रेष्ठ रात के समय अद्वार से कोट को फांद कर का में प्रविष्ट हृया \nधानरराज के उस हितैषी ने लंका नगरी में प्रवेश करके मानों अपना बायां पाओं शत्रु के सिर पर रख दिया \nउस समय सुन्दर \'सब ओर से सजे हुए श्वेत मेघ के तुल्य राक्षसों के जो पद्माकार, स्वस्तिकाकार, और वर्धमान घर है, उन से इंका जगमन. कर रही थी, राधा के अर्थ वह श्रीमान् घूमता हुमा उसे देखता भया और आनन्दित होता भया ।। ३, ४ \nएक भवन से दूसरे भवन को जाते हुए उस वानरश्रेष्ठ ने वहाँ र विविध भाकृति और रूपों वाले भवन देखे \nवहाँ राक्षसों के घरों में उसने अप "करते हुओं के मन्त्र मुने और स्वाध्याय में रत राक्षसों को देखा \nराक्षसों के घर से घर और बगीचों को देखता हुआ बेधड़क यह महलों के पास घूमा \nतव पवनपुत्र हनुमान ने यह भवन श्रेष्ठ देखा, जो राक्षसपति का भवन है, और बहुत महलों से भरपूर है \nविशाल नेत्रोंवाली वैदेही सीता को ढूंढता हुआ शत्रुनों के मारनेवाला, हनुमान उसके चारों ओर घूमा ।। ९ \nतब 

In [13]:
summarize(str_text[0:2500])

"घ४-९) सीता को रावण के भन्तापुर में ढूंढना : वह महावीर्य महान हृदय पाछा वानर श्रेष्ठ रात के समय अद्वार से कोट को फांद कर का में प्रविष्ट हृया \nउस समय सुन्दर 'सब ओर से सजे हुए श्वेत मेघ के तुल्य राक्षसों के जो पद्माकार, स्वस्तिकाकार, और वर्धमान घर है, उन से इंका जगमन.\nराक्षसों के घर से घर और बगीचों को देखता हुआ बेधड़क यह महलों के पास घूमा \nतब वह उस सुन्दर पड़ी माला की ओर मस्थित हुआ, जो उत्तम स्त्री की तरह रावण की बड़ी पारी थी \nमें बस वानर यूयपति ने चन्द्र तुल्य मुखवाली, सुन्दर कुण्डल पहने हुई, ताजे घुषषों की पालाएं और भूषणोंवाली पनियों को देसा "

In [14]:
len(str_text)

68718

In [15]:
df = pd.DataFrame()

In [16]:
def summarization(str_text):
    startflag = 0
    endflag = 2500
    parsed_text = {'text':[], 'title':[]}
    for summary in range(int(len(str_text)/endflag)):
        summarization_block = summarize(str_text[startflag:endflag])
        parsed_text['text'].append(str_text[startflag:endflag])
        parsed_text['title'].append(summarization_block)
        startflag +=2500
        endflag +=2500
    return pd.DataFrame(parsed_text)

In [17]:
df = summarization(str_text)
df.shape

(27, 2)

In [18]:
#df.to_csv('input_abstarct_summary.csv')

### Alternative Summarization from Gensim 

In [19]:
def textrankTfIdf(document):
    # sentence_tokenizer = PunktSentenceTokenizer()
    # sentences = sentence_tokenizer.tokenize(document, 'hindi')

    sentences = document
    bow_matrix = CountVectorizer().fit_transform(sentences)
    # normalized = TfidfTransformer(norm='l2', use_idf=True, use_bm25idf=True, smooth_idf=True,
    #              delta_idf=False, sublinear_tf=False, bm25_tf=True).fit_transform(bow_matrix)

    normalized = TfidfTransformer().fit_transform(bow_matrix)
    similarity_graph = normalized * normalized.T

    nx_graph = nx.from_scipy_sparse_matrix(similarity_graph)
    scores = nx.pagerank(nx_graph)
    return sorted(((scores[i], s) for i, s in enumerate(sentences)),
                  reverse=True)
def orderSentences(rankedList, data, initSentences):
    index = ['']*len(data)
    # print(rankedList)
    for eachRanked in rankedList[0:int(math.ceil(0.2*len(rankedList)))]:
        sen = eachRanked[1]
        index[data.index(sen)] = initSentences[data.index(sen)]
        # print(data.index(sen))
    return index

In [20]:
int(len(text_sentence)/25)

26

In [21]:
def alternate_summarization(text_sentence):
    startflag = 0
    endflag = 25
    parsed_text = {'text':[], 'alt_title':[]}
    sentence =''
    for summary in range(int(len(text_sentence)/endflag)):
        rankedSentences = textrankTfIdf(text_sentence[startflag:endflag])
        orderedsentences = orderSentences(rankedSentences, text_sentence[startflag:endflag], text_sentence[startflag:endflag])
        for ordered in orderedsentences:
            if ordered != "":
                sentence += ordered 
        parsed_text['alt_title'].append(sentence)
        parsed_text['text'].append(text_sentence[startflag:endflag])
        startflag +=25
        endflag +=25
        sentence =''
    return pd.DataFrame(parsed_text)


In [22]:
df2 = alternate_summarization(text_sentence)

In [23]:
df2.head()

,text,alt_title
0,[ वानर हनुमान सीमा के देखने की उत्कण्ठा येसूया...,उस समय सुन्दर 'सब ओर से सजे हुए श्वेत मेघ के ...
1,[ मेरी दृष्टि आजतक (ऐमी अवस्था में ) परस्त्रिय...,सो मैंने शुद्ध मन से रावण का पारा अन्तःपुर ढू...
2,[ तब उस वानर ने सारे जगह में सुहावना एक पर्वत ...,तब उस वानर ने सारे जगह में सुहावना एक पर्वत द...
3,"[ , १८) प्रभात का समय और रावण का अशोकवनिका में...",उस समय मङ्गक बाओं और कानों के प्यारे शब्दों स...
4,"[ पह तेरा सुन्दर बना हुथा, यौवन चला जारहा है, ...",सारे लोकों से पल से हर कर जो मैं रत्न लाचा हू...


In [76]:
df2[['text','alt_title']].head(1)

,text,alt_title
0,[ वानर हनुमान सीमा के देखने की उत्कण्ठा येसूया...,उस समय सुन्दर 'सब ओर से सजे हुए श्वेत मेघ के ...


In [24]:
#df2.to_csv('summary_alt_gensim.csv')

In [25]:
#df2 = pd.read_csv('../data/summary_alt_gensim.csv')

In [26]:
def extract_ner(sentence_ner,row_no):
    parsed_text = {'sentence':[], 'word':[], 'ner':[]}
    words = re.findall("([^<]+)<[^>]+>",sentence_ner)
    tags = re.findall("<([^>]+)>",sentence_ner)
    
    # for word in words: append word #we need to write seperate tags
    for index in range(len(words)):
        parsed_text['word'].append(words[index])
        parsed_text['ner'].append(tags[index])
        parsed_text['sentence'].append(row_no)
    
    return pd.DataFrame(parsed_text)

In [27]:
from flair.data import Sentence
from flair.models import SequenceTagger
# load the model you trained
model = SequenceTagger.load('/Users/rohanraj/Documents/Master/CaseStudy2/code/resources/taggers/netag/best-model.pt')

df_ner = pd.DataFrame()

r, c = df2.shape
for row_no in range(r):
    sentence = Sentence(df2['alt_title'][row_no])
    model.predict(sentence)
    sentence_ner = sentence.to_tagged_string()
    temp_df = extract_ner(sentence_ner,row_no)
    df_ner = df_ner.append(temp_df,ignore_index=True)  
# predict tags and print
#print(sentence.to_tagged_string())


2020-03-04 21:23:43,200 loading file /Users/rohanraj/Documents/Master/CaseStudy2/code/resources/taggers/netag/best-model.pt


In [28]:
df_ner.head()

,sentence,word,ner
0,0,उस,o
1,0,समय,o
2,0,सुन्दर,B-PERSON
3,0,'सब,B-LOCATION
4,0,ओर,B-PERSON


In [29]:
#df_ner.to_csv('../data/ner_tag.csv')

In [30]:
#df_ner = pd.read_csv('../data/ner_tag.csv',index_col=0)

In [31]:
df_ner.head()

,sentence,word,ner
0,0,उस,o
1,0,समय,o
2,0,सुन्दर,B-PERSON
3,0,'सब,B-LOCATION
4,0,ओर,B-PERSON


In [32]:
# Get names of indexes for which column Age has value 30
df_ner_imp = df_ner[ df_ner['ner'].str.endswith('PERSON') | df_ner['ner'].str.endswith('LOCATION') ]
# Delete these row indexes from dataFrame
#df_ner.drop(indexNames , inplace=True)

In [33]:
df_ner_imp.head()

,sentence,word,ner
2,0,सुन्दर,B-PERSON
3,0,'सब,B-LOCATION
4,0,ओर,B-PERSON
8,0,श्वेत,B-PERSON
9,0,मेघ,B-PERSON


In [34]:
#df_ner_imp.to_csv('../data/ner_tag_imp.csv')

In [35]:
#df_ner.to_csv('../data/ner_tag.csv')

In [36]:
# Standford nlp is being used for lemmatization

In [37]:
def lemmatization(text):
    lemmatized_text = []
    for line in tqdm(text):
        if line not in [""," "] :
            doc = nlp(line)
            for sent in doc.sentences:
                for wrd in sent.words:
                    #extract text and lemma
                    lemmatized_text.append(wrd.lemma)
    return lemmatized_text

def remove_stopwords(word_tokenized,stop_words):
    return [word for word in word_tokenized if word not in stop_words]

def custom_remove_garbage(original_words_list,list_of_garbage_words):
    tmp_list = [word for word in original_words_list if word not in list_of_garbage_words] # garbage list
    tmp_list = [word for word in tmp_list if len(re.findall("\d+",word))==0] # english numbers
    tmp_list = [word for word in tmp_list if len(re.findall("[a-zA-Z]+",word))==0] # english alphabets
    return tmp_list

def Diff(li1, li2): 
    li_dif = [i for i in li1 + li2 if i not in li1 or i not in li2] 
    return li_dif 

## Topic Meddling after prepocessing

In [38]:
def topic_modelling(text_sentence,stop_words):
    df = pd.DataFrame()
    startflag = 0
    endflag = 25
    for text in range(int(len(text_sentence)/endflag)):
        
        #Clean text after lemmatization
        lemmatized = lemmatization(text_sentence[startflag:endflag])
        clean_text = remove_stopwords(lemmatized,stop_words)
        #len(clean_text),len(lemmatized) #281,612
        
        # remove punctuation from each word
        table = str.maketrans('', '', string.punctuation)
        stripped = [w.translate(table) for w in clean_text]
        #print(stripped[:25])
        #print(len(clean_text),len(stripped)) #281

        resultant = Diff(clean_text,stripped)
        #len(set(resultant)) #19

        final_text = custom_remove_garbage(stripped,resultant)
        #len(final_text) #265
        
        # Create Dictionary
        id2word = corpora.Dictionary([final_text])

        # Create Corpus
        texts = final_text

        # Term Document Frequency
        corpus = [id2word.doc2bow(texts)] 
        
        # Build LDA model

        lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=1, 
                                           random_state=100,
                                           update_every=10,
                                           chunksize=100,
                                           passes=20,
                                           alpha='auto',
                                           per_word_topics=True)
        #pprint(lda_model.print_topics())
        temp_file = datapath("model")
        lda_model.save(temp_file)
        lda = gensim.models.ldamodel.LdaModel.load(temp_file)
        lda.update(corpus)
        lda.save('model')
        temp_df = pd.DataFrame(lda.print_topics())
        df = df.append(temp_df,ignore_index=True) 
        startflag +=25
        endflag +=25
    return df

In [39]:
df4 = topic_modelling(text_sentence,stop_words)

100%|██████████| 25/25 [00:02<00:00,  9.70it/s]


In [40]:
df4.shape

(26, 2)

In [41]:
df4.tail(5)

,0,1
21,0,"0.025*""कह"" + 0.019*""वाक्य"" + 0.013*""विभीषण"" + ..."
22,0,"0.018*""वाक्य"" + 0.011*""भाई"" + 0.011*""विभीषण"" +..."
23,0,"0.024*""कह"" + 0.024*""सब"" + 0.021*""राम"" + 0.017*..."
24,0,"0.025*""राम"" + 0.017*""समुद्र"" + 0.014*""रावण"" + ..."
25,0,"0.029*""सेना"" + 0.024*""सारण"" + 0.021*""कह"" + 0.0..."


In [42]:
#df4.to_csv('combined_model_topic.csv')

In [43]:
temp_file = datapath("model")
lda = gensim.models.ldamodel.LdaModel.load(temp_file)

In [44]:
#df4 =pd.read_csv('../data/combined_model_topic.csv',index_col=0)

In [52]:
df4.head()

,0,1
0,0,"0.017*""भवन"" + 0.015*""सीता"" + 0.012*""राक्षस"" + ..."
1,0,"0.015*""हनुमान"" + 0.013*""युक्त"" + 0.013*""सब"" + ..."
2,0,"0.027*""राम"" + 0.016*""सीता"" + 0.013*""वानर"" + 0...."
3,0,"0.027*""रावण"" + 0.015*""सीता"" + 0.011*""भूषण"" + 0..."
4,0,"0.014*""पा"" + 0.011*""सीता"" + 0.009*""वचन"" + 0.00..."


In [56]:
df4['1']

KeyError: '1'

In [51]:
df4.columns

RangeIndex(start=0, stop=2, step=1)

In [55]:
re.findall('[!@#$%^&*(),.?":{}|<>]',df4[1][0])

['.',
 '*',
 '"',
 '"',
 '.',
 '*',
 '"',
 '"',
 '.',
 '*',
 '"',
 '"',
 '.',
 '*',
 '"',
 '"',
 '.',
 '*',
 '"',
 '"',
 '.',
 '*',
 '"',
 '"',
 '.',
 '*',
 '"',
 '"',
 '.',
 '*',
 '"',
 '"',
 '.',
 '*',
 '"',
 '"',
 '.',
 '*',
 '"',
 '"']

In [58]:
re.sub('[!@#$%^&*(),.?"+:{}|<>0-9]', '', df4[1][0])

'भवन  सीता  राक्षस  सुन्दर  भया  रावण  वानर  सक्ती  मा  स्त्री'

In [ ]:
df4['1'][0]

In [ ]:
df4.iloc[0]

In [59]:
def process_topic(df):
    parsed_text = {'top_topic':[]}
    
    for ind in df.index:
        parsed_text['top_topic'].append(re.sub('[!@#$%^&*(),.?"+:{}|<>0-9]', '', df[1][ind]))
    return pd.DataFrame(parsed_text)    

In [60]:
topic_df= process_topic(df4)

In [68]:
topic_df['top_topic'][25]

'सेना  सारण  कह  राम  वानर  राक्षस  गुप्तचर  बोल  शुक  विभीषण'

In [62]:
topic_df.to_csv('combined_model_topic_top_clean.csv')